In [1]:
from scipy.spatial import KDTree
from geopy.distance import geodesic
import pandas as pd

In [2]:
gh_df = (
    pd.read_csv("geohash_traffic_density_pt_15clusters.csv", usecols=['GEOHASH', 'LATITUDE', 'LONGITUDE'])
)

# Reading Minibus stops data and converting to dictionary
site_dict = (
    pd.read_csv("datasets/10_ist_minibus_stops/tr_ist_minibus_stands.csv")
    .set_index('name')
    .T
    .to_dict()
)

In [3]:
# Unique list of coordinates
coordinates = gh_df[['LATITUDE', 'LONGITUDE']].drop_duplicates().values
coordinates

array([[41.080627, 28.811646],
       [40.987244, 29.108276],
       [41.003723, 29.09729 ],
       ...,
       [40.976257, 29.229126],
       [41.009216, 27.98767 ],
       [41.020203, 27.998657]])

In [4]:
def find_nearest_coordinates(
        target_lat: float,
        target_lon: float,
        coordinates: list[list[float, float]]
        ) -> list[tuple[tuple[float, float], float]]:
    """
    Finds the nearest coordinates to a target latitude and longitude.

    Args:
        target_lat (float): The latitude of the target location.
        target_lon (float): The longitude of the target location.
        coordinates (List[Tuple[float, float]]): A list of tuples representing the coordinates to search from.

    Returns:
        List[Tuple[Tuple[float, float], float]]: A list of tuples containing the nearest coordinates and their distances in kilometers.
    """
    target_coords = (target_lat, target_lon)
    
    # Build a KDTree for efficient nearest neighbor search
    tree = KDTree(coordinates)

    # Calculate the search radius in degrees
    radius_deg = 1 / 111.0  # 1 km radius in degrees

    # Query the KDTree for indices of points within the search radius
    indices = tree.query_ball_point([(target_lat, target_lon)], radius_deg)

    # Calculate the distance between target coordinates and nearest coordinates
    nearest_coordinates = [(coordinates[i], geodesic(target_coords, coordinates[i]).km) for i in indices[0]]

    # Sort the nearest coordinates by distance
    nearest_coordinates.sort(key=lambda x: x[1])  # Sort by distance

    return nearest_coordinates

In [5]:
mb_list = []
new_list = []

for site in site_dict:

    target_lat = site_dict[site]['latitude']
    target_lon = site_dict[site]['longitude']

    nearest_coords = find_nearest_coordinates(target_lat, target_lon, coordinates)

    for i, (coords, distance) in enumerate(nearest_coords):

        gh = gh_df.loc[gh_df['LATITUDE'].eq(coords[0]) & gh_df['LONGITUDE'].eq(coords[1]), 'GEOHASH'].values[0]

        # Creating two dataframe which will later be merged rows wise for visualization purpose.
        mb_list.append({
            "type" : "minibus_stops",
            "name" : site,
            "latitude" : target_lat,
            "longitude" : target_lon,
            "nearest" : site
        })

        new_list.append({
            "type" : "geohash",
            "name" : gh,
            "latitude" : coords[0],
            "longitude" : coords[1],
            "nearest" : site
        })

In [6]:
df = (
    pd.concat([pd.DataFrame(new_list), pd.DataFrame(mb_list)], ignore_index=True)
    .drop_duplicates(ignore_index=True)
)

df

,type,name,latitude,longitude,nearest
0,geohash,sxk9vh,41.113586,29.053345,kml_1
1,geohash,sxk9v5,41.108093,29.053345,kml_1
2,geohash,sxk9v1,41.097107,29.053345,kml_3
3,geohash,sxk9v0,41.091614,29.053345,kml_3
4,geohash,sxk9sp,41.086120,29.009400,kml_4
...,...,...,...,...,...
2594,minibus_stops,kml_689,41.072118,28.263787,kml_689
2595,minibus_stops,kml_690,41.073037,28.264155,kml_690
2596,minibus_stops,kml_691,41.093108,28.153423,kml_691
2597,minibus_stops,kml_700,41.026261,29.022258,kml_700


In [7]:
# Soem GEOHASH as up to 27 minibus stops within its 1km radius
(
    df.query("type=='geohash'")
    .groupby('name')['nearest'].nunique()
    .nlargest(10)
)

name
sxk1ej    27
sxk93g    27
sxk964    24
sxk965    24
sxk96h    24
sxk915    23
sxk91h    23
sxk3nt    22
sxk3nw    22
sxk93f    21
Name: nearest, dtype: int64

In [8]:
df.to_csv("datasets/13_gh_proximities/gh_nearby_minibus_stops.csv", index=False)